In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import Model
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, UpSampling2D
from keras.callbacks import ModelCheckpoint

from mne.io import read_raw_brainvision as mne_read
from os.path import join
from os import listdir
import numpy as np

Using TensorFlow backend.


In [3]:
channels_num = 60
window_size = 500
nb_epoch = 50

# Hyper parameters
batch_size = 32
nb_visible = channels_num * window_size
nb_hidden = 1000
nb_emb = 20

corruption_level = 0.001

In [4]:
mne_format_data_dir = join('data', 'resting_state_mne_format')

In [5]:
def create_model():
    inp = Input(shape=(window_size, channels_num, 1))

    encoder = Sequential( [
        Conv2D(256, kernel_size=(3,3), activation='relu', padding='same', input_shape=(window_size,channels_num,1)),
        MaxPooling2D(),
        Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'),
        #MaxPooling2D(),
        Conv2D(1, kernel_size=(1,1), padding='same'),
    ] )
    print(encoder.summary())
    decoder = Sequential((
        Conv2D(256, kernel_size=(3,3), activation='relu', padding='same', input_shape=(125, 15, 1) ),
        UpSampling2D(),
        Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'),
        #UpSampling2D(),
        Conv2D(1, kernel_size=(1,1), padding='same'),
    ))
    print(decoder.summary())
    autoencoder = Model(inp, decoder(encoder(inp)))
    autoencoder.compile(loss='mse', optimizer='adam', metrics=['mae'])

    print(autoencoder.summary())
    return autoencoder

In [6]:
autoencoder = create_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 500, 60, 256)      2560      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 250, 30, 256)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 250, 30, 128)      295040    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 250, 30, 1)        129       
Total params: 297,729
Trainable params: 297,729
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 125, 15, 256)      2560      
_________________________________________________________________

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
#scaler = StandardScaler()
mean, var = None, None

def cut_window(data):
    return data[:, :data.shape[1] - data.shape[1] % window_size]

def read_func(filename, verbose=0):
    d = mne_read(filename, verbose=verbose, preload=True)
    d.drop_channels(['VEOG', 'HEOG', 'STI 014'])
    d = d.get_data()
    return cut_window(np.vstack( [d, np.zeros( (2, d.shape[1]) )]) )

def read_data(data_type, return_batch_np=False):
    global mean, var
    ''' If return_batch_np is False
            return data with shape (None, batch_size, nb_visible)
        else number of batches in data
    '''
    verbose = None if return_batch_np else 0
    
    data_dir = join(mne_format_data_dir, data_type)
    data = np.hstack([read_func( join(data_dir, fn), verbose=verbose) for fn in listdir(data_dir) if fn[-5:] == '.vhdr'])
    
    if mean is None:
        mean, var = data.mean(), data.var()
    #data = scaler.transform(data)
    data = (data - mean) / (var ** (1/2))
    print(data_type, data.mean(), data.var())
    if return_batch_np:
        return data.shape[1] // window_size // batch_size
    return data.T.reshape(-1, window_size, channels_num, 1)

def gen(data_type):
    data = read_data(data_type)
    print(data.shape)
    for ep in range(nb_epoch+1):
        for i in range(data.shape[0] // batch_size):
            x = data[i*batch_size:(i+1)*batch_size, :, :, :]
            yield x, x

In [11]:
train_batches_nb = read_data('train', return_batch_np=True)
val_batches_nb = read_data('validation', return_batch_np=True)

Extracting parameters from data/resting_state_mne_format/train/2505_shirokova_post_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 606599  =      0.000 ...   606.599 secs...
Extracting parameters from data/resting_state_mne_format/train/miloslavov_22_05_pre_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 603399  =      0.000 ...   603.399 secs...
Extracting parameters from data/resting_state_mne_format/train/gorin_rest_eeg_post_31011200.vhdr...
Setting channel info structure...
Reading 0 ... 603699  =      0.000 ...   603.699 secs...
Extracting parameters from data/resting_state_mne_format/train/2704_zagirova_post_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 604549  =      0.000 ...   604.549 secs...
Extracting parameters from data/resting_state_mne_format/train/2103_glebko_posteeg.vhdr...
Setting channel info structure...
Reading 0 ... 633899  =      0.000 ...   633.899 secs...
Extracting parameters from data/resting_state_mne_format/train/200317

In [14]:
import matplotlib.pyplot as plt
from keras.models import load_model
autoencoder = load_model('models/cnn_x2_00.hdf5') # обучение прерывалось, в итоге проведено 3 эпохи

In [13]:
history = autoencoder.fit_generator(gen('train'), steps_per_epoch=train_batches_nb, epochs=nb_epoch, \
                          validation_data=gen('validation'), validation_steps=val_batches_nb, \
                          #use_multiprocessing=True, \
                          callbacks=[ModelCheckpoint("models/cnn_%s_{epoch:02d}.hdf5" % ('x2'), save_best_only=False)])

Epoch 1/50
train -6.34378883316e-16 1.0
(29879, 500, 60, 1)
932/933 [============================>.] - ETA: 8s - loss: 0.0353 - mean_absolute_error: 0.1369 validation -0.164368573828 0.135983946103
(3149, 500, 60, 1)
933/933 [==============================] - 8357s - loss: 0.0353 - mean_absolute_error: 0.1369 - val_loss: 0.0507 - val_mean_absolute_error: 0.1790
Epoch 2/50
933/933 [==============================] - 8316s - loss: 0.0418 - mean_absolute_error: 0.1391 - val_loss: 0.0594 - val_mean_absolute_error: 0.1889
Epoch 3/50
275/933 [=======>......................] - ETA: 5701s - loss: 0.0309 - mean_absolute_error: 0.1334

KeyboardInterrupt: 

In [15]:
test_batches_nb = read_data('test', return_batch_np=True)

Extracting parameters from data/resting_state_mne_format/test/zavrin_eyes_closed_eeg_15021500.vhdr...
Setting channel info structure...
Reading 0 ... 302749  =      0.000 ...   302.749 secs...
Extracting parameters from data/resting_state_mne_format/test/zavrin_15021500_eyesclosed_post_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 319299  =      0.000 ...   319.299 secs...
Extracting parameters from data/resting_state_mne_format/test/300120171600_dagaev_rest_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 615599  =      0.000 ...   615.599 secs...
Extracting parameters from data/resting_state_mne_format/test/tsoy_pre_eeg_2504.vhdr...
Setting channel info structure...
Reading 0 ... 652699  =      0.000 ...   652.699 secs...
test -0.102788369207 0.114159913247


In [12]:
import matplotlib.pyplot as plt
from keras.models import load_model
autoencoder = load_model('models/cnn_x2_01.hdf5')

In [16]:
def metric(y_true, y_pred):
    l2ratio = np.sum((y_true - y_pred)**2) / np.sum(y_true**2)
    return np.sqrt(l2ratio) * 100

In [ ]:
x = []
i = 0
for test in gen('test'):
    i += 1
    x.append(metric(autoencoder.predict(test[0]), test[1]))
    if (i % 25) == 0:
        print(np.mean(x))

test -0.102788369207 0.114159913247
(3779, 500, 60, 1)
44.1733951114
54.7072529972
63.6165421966
61.725480184
59.5827152153
57.4937336196
59.8050138551
61.8361885811
60.952260958
59.5843298893
58.8064642985
60.6772702879
61.229790427
60.7395522172
59.6786332029
59.7376498832
61.006266672
60.9694207975
60.5546350934
59.7966718883
60.2238764893
61.1589938855
60.8270312745
60.3523191617
59.8702304584
60.5505744035
60.9866933048
60.7410139377
60.2200398377
60.0726178894
60.7535601738
60.8755723587
60.6845202583
60.2037934972
60.3148207207
60.9012588937
60.7983268406
60.5545173129
60.1975504269
60.5031575545
60.9118138075
60.7423870775
60.4418370054
60.1936641596
60.6602851411
60.8401670087
60.701130234
60.3611244101
60.3545488598
60.7731659643
60.7860158591
60.6479607499
60.3524616656
60.4860242374
60.8650042497
60.7436775865
60.55454072
60.3361463372
60.6063185052
60.8219629535
